## **1. The World Bank's international debt data**
In this notebook we are going to analyze debt data collected by The World Bank. The dataset contains information about the amount of debt owed by developing countries and we are particularly interested in the following questions:

> - What is the total amount of debt that is owed by the countries listed in the dataset? <br>
> - Which country owns the maximum amount of debt and what does that amount look like? <br>
> - What is the average amount of debt owed by countries across different debt indicators? <br>

Let us sneak peek into the dataset and see what it looks like but first we need to correcly open the dataframe to be able to make the queries, this time we will use SQL through Python using the pertinent libraries.

In [25]:
import pandas as pd
import sqlite3 as sql

international_debt = pd.read_csv('dataset/international_debt.csv')
conn = sql.connect('default.db')
df.to_sql('international_debt', conn)

query = 'select * from df'
new_df = pd.read_sql(query, conn)
new_df.head(3)


,index,country_name,country_code,indicator_name,indicator_code,debt
0,0,Afghanistan,AFG,"Disbursements on external debt, long-term (DIS...",DT.DIS.DLXF.CD,72894453.7
1,1,Afghanistan,AFG,"Interest payments on external debt, long-term ...",DT.INT.DLXF.CD,53239440.1
2,2,Afghanistan,AFG,"PPG, bilateral (AMT, current US$)",DT.AMT.BLAT.CD,61739336.9


Now let us see the first 10 rows of the dataset.

In [36]:
# Create the query to select all the columns from the df1 table
query = 'SELECT * FROM international_debt LIMIT 10'
df1 = pd.read_sql(query, conn)
print(df1.head)

<bound method NDFrame.head of    index country_name country_code  \
0      0  Afghanistan          AFG   
1      1  Afghanistan          AFG   
2      2  Afghanistan          AFG   
3      3  Afghanistan          AFG   
4      4  Afghanistan          AFG   
5      5  Afghanistan          AFG   
6      6  Afghanistan          AFG   
7      7  Afghanistan          AFG   
8      8  Afghanistan          AFG   
9      9  Afghanistan          AFG   

                                      indicator_name  indicator_code  \
0  Disbursements on external debt, long-term (DIS...  DT.DIS.DLXF.CD   
1  Interest payments on external debt, long-term ...  DT.INT.DLXF.CD   
2                  PPG, bilateral (AMT, current US$)  DT.AMT.BLAT.CD   
3                  PPG, bilateral (DIS, current US$)  DT.DIS.BLAT.CD   
4                  PPG, bilateral (INT, current US$)  DT.INT.BLAT.CD   
5               PPG, multilateral (AMT, current US$)  DT.AMT.MLAT.CD   
6               PPG, multilateral (DIS, current

## **2. Finding the number of distinct countries**
We can see that there is more than one debt indicator in the dataset. Let us find out the number of distinct debt indicators in the dataset and also the number of distinct countries so we want to deal with this unique values.

In [35]:
# Create the pertinent query
query = 'SELECT COUNT(DISTINCT(country_name)) AS total_distinct_countries FROM international_debt'
df2 = pd.read_sql(query, conn)
print(df2.head)

<bound method NDFrame.head of    total_distinct_countries
0                       124>


## **3. About debt**
As before we want to find out the distinct debt indicators in the dataset but first to get a better overview we want to see the total debt owed.

In [34]:
#Create the pertinent query
query = 'SELECT ROUND(SUM(debt)/1000000, 2) AS total_debt FROM international_debt '
df3 = pd.read_sql(query, conn)
print(df3)

   total_debt
0  3079734.49


In [33]:
# Create the pertinent query
query = '''SELECT country_name, SUM(debt) AS total_debt FROM international_debt 
        GROUP BY country_name  ORDER BY total_debt DESC LIMIT 1'''
df4 = pd.read_sql(query, conn)
print(df4)

  country_name    total_debt
0        China  2.857935e+11


## **4. Average amount of debt by indicator**

So now we have a better idea of the dataset and a few of its statistics now lets see the average amount of debt across different debt indicators or countries.

In [32]:
# Create the pertinent query
query = ''' SELECT 
    indicator_code AS debt_indicator,
    indicator_name,
    AVG(debt) AS average_debt
    FROM international_debt
    GROUP BY debt_indicator, indicator_name
    ORDER BY average_debt DESC
    LIMIT 10 '''
df5 = pd.read_sql(query, conn)
print(df5)

   debt_indicator                                     indicator_name  \
0  DT.AMT.DLXF.CD  Principal repayments on external debt, long-te...   
1  DT.AMT.DPNG.CD  Principal repayments on external debt, private...   
2  DT.DIS.DLXF.CD  Disbursements on external debt, long-term (DIS...   
3  DT.DIS.OFFT.CD         PPG, official creditors (DIS, current US$)   
4  DT.AMT.PRVT.CD          PPG, private creditors (AMT, current US$)   
5  DT.INT.DLXF.CD  Interest payments on external debt, long-term ...   
6  DT.DIS.BLAT.CD                  PPG, bilateral (DIS, current US$)   
7  DT.INT.DPNG.CD  Interest payments on external debt, private no...   
8  DT.AMT.OFFT.CD         PPG, official creditors (AMT, current US$)   
9  DT.AMT.PBND.CD                      PPG, bonds (AMT, current US$)   

   average_debt  
0  5.904868e+09  
1  5.161194e+09  
2  2.152041e+09  
3  1.958983e+09  
4  1.803694e+09  
5  1.644024e+09  
6  1.223139e+09  
7  1.220411e+09  
8  1.191188e+09  
9  1.082624e+09  


We can see that the indicator *DT.AMT.DLXF.CD* has the highest average debt amount. This includes repayment of long terms debt, Another interesting observation that can be done is that there is a HUGE difference between the indicators so lets do a little more analysis on this indicator.

## **5. The most common debt indicator**

In [37]:
# Create the pertinent query
query = ''' SELECT 
    country_name, 
    indicator_name
    FROM international_debt
    WHERE debt = (SELECT 
    MAX(debt)
    FROM international_debt
    WHERE indicator_code = 'DT.AMT.DLXF.CD') '''
df6 = pd.read_sql(query, conn)
print(df6)

  country_name                                     indicator_name
0        China  Principal repayments on external debt, long-te...


As we already knew China has the highest debt amount but lets see which country has the highest debt amount for the indicator *DT.AMT.DLXF.CD*. We saw that long-term debt is the most common debt indicator so lets see which country has the highest long-term debt amount.

Now, what about the countries in general average debt.

In [38]:
# Create the pertinent query
query = '''SELECT 
        indicator_code,
        COUNT(indicator_code) AS indicator_count
        FROM international_debt
        GROUP BY indicator_code
        ORDER BY indicator_count DESC, indicator_code DESC
        LIMIT 20'''
df7 = pd.read_sql(query, conn)
print(df7)

    indicator_code  indicator_count
0   DT.INT.OFFT.CD              124
1   DT.INT.MLAT.CD              124
2   DT.INT.DLXF.CD              124
3   DT.AMT.OFFT.CD              124
4   DT.AMT.MLAT.CD              124
5   DT.AMT.DLXF.CD              124
6   DT.DIS.DLXF.CD              123
7   DT.INT.BLAT.CD              122
8   DT.DIS.OFFT.CD              122
9   DT.AMT.BLAT.CD              122
10  DT.DIS.MLAT.CD              120
11  DT.DIS.BLAT.CD              113
12  DT.INT.PRVT.CD               98
13  DT.AMT.PRVT.CD               98
14  DT.INT.PCBK.CD               84
15  DT.AMT.PCBK.CD               84
16  DT.INT.DPNG.CD               79
17  DT.AMT.DPNG.CD               79
18  DT.INT.PBND.CD               69
19  DT.AMT.PBND.CD               69


## **6. Other viable debt issues and conclusion**

So there a total of 6 debt indicators in which the countries have taken debt there is cleary some more question regading the indicators, but now we can also focus in de debts and the countries separately.

In [39]:
# Create the pertinent query
query = '''SELECT 
        indicator_code,
        COUNT(indicator_code) AS indicator_count
        FROM international_debt
        GROUP BY indicator_code
        ORDER BY indicator_count DESC, indicator_code DESC
        LIMIT 20'''
df8 = pd.read_sql(query, conn)
print(df8)

    indicator_code  indicator_count
0   DT.INT.OFFT.CD              124
1   DT.INT.MLAT.CD              124
2   DT.INT.DLXF.CD              124
3   DT.AMT.OFFT.CD              124
4   DT.AMT.MLAT.CD              124
5   DT.AMT.DLXF.CD              124
6   DT.DIS.DLXF.CD              123
7   DT.INT.BLAT.CD              122
8   DT.DIS.OFFT.CD              122
9   DT.AMT.BLAT.CD              122
10  DT.DIS.MLAT.CD              120
11  DT.DIS.BLAT.CD              113
12  DT.INT.PRVT.CD               98
13  DT.AMT.PRVT.CD               98
14  DT.INT.PCBK.CD               84
15  DT.AMT.PCBK.CD               84
16  DT.INT.DPNG.CD               79
17  DT.AMT.DPNG.CD               79
18  DT.INT.PBND.CD               69
19  DT.AMT.PBND.CD               69


In this notebook, we took a look at debt owed by countries across the globe. We extracted a few summary statistics from the data and unraveled some interesting facts and figures. We also validated our findings to make sure the investigations are correct.

#### *This was collected and solved by jdpalmad and the project was guided by Datacamp, the dataset is from The Worlds Bank*